In [8]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
from ccfj import GetStationPairs
import pandas as pd
import yaml
import geopandas as gp
import openpyxl
from openpyxl import load_workbook
from concurrent.futures import ThreadPoolExecutor
from scipy.interpolate import make_interp_spline

In [9]:
flag_project = 1 # 0--regular ; 1--repartrition

In [10]:
if flag_project == 0:
    file_project = 'a-project.yml'
    with open('a-project.yml', 'r', encoding='utf-8') as f:
        proj = yaml.load(f.read(), Loader=yaml.FullLoader)
    name_project = proj['name']
    #name_project = 'project/output_FJSJ_16-01/'               # Harmon server
elif flag_project == 1:
    file_project = 'a-project_repar.yml'
    with open('a-project_repar.yml', 'r', encoding='utf-8') as f:
        proj = yaml.load(f.read(), Loader=yaml.FullLoader)
    name_project = proj['name']
    #name_project = 'project_repartrition/repartrition_01-03/'               # Harmon server


In [11]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  ./
dir_project_workspace:  ./
dir_project:  ./project_repartition_v4.0/output_repar_v9.2_01-01/


In [12]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [13]:
dir_disp_autopick = dir_project + info_basic['dir_disp']

In [14]:
key_subworks = info_basic['key_subworks']
#key_subworks = info_basic['key_subworks_repick']
key_subworks

['1--08-03',
 '2--09-03',
 '3--10-03',
 '4--11-03',
 '5--12-03',
 '6--13-03',
 '7--14-03',
 '8--15-03',
 '9--16-03',
 '10--18-03',
 '11--19-03',
 '12--20-03',
 '13--21-03',
 '14--22-03',
 '15--23-03',
 '16--24-03',
 '17--25-03',
 '18--26-03',
 '19--27-03',
 '20--28-03',
 '21--29-03',
 '22--30-03',
 '23--31-03',
 '24--32-03',
 '25--33-03',
 '26--34-03',
 '27--35-03',
 '28--36-03',
 '29--37-03',
 '30--41-03',
 '31--42-03',
 '32--43-03',
 '33--46-03',
 '34--47-03',
 '35--48-03',
 '36--52-03',
 '37--53-03',
 '38--07-04',
 '39--08-04',
 '40--09-04',
 '41--10-04',
 '42--12-04',
 '43--13-04',
 '44--17-04',
 '45--20-04',
 '46--21-04',
 '47--22-04',
 '48--23-04',
 '49--24-04',
 '50--25-04',
 '51--27-04',
 '52--28-04',
 '53--29-04',
 '54--30-04',
 '55--31-04',
 '56--32-04',
 '57--33-04',
 '58--34-04',
 '59--35-04',
 '60--36-04',
 '61--41-04',
 '62--42-04',
 '63--47-04',
 '64--48-04',
 '65--49-04',
 '66--52-04',
 '67--53-04',
 '68--08-05',
 '69--12-05',
 '70--13-05',
 '71--14-05',
 '72--16-05',
 

### create workspace for Dispernet

In [15]:
info_basic['dir_inv_dispernet'] = 'inv_dispernet/'
dir_inv = dir_project + info_basic['dir_inv_dispernet']
if not os.path.exists(dir_inv):
    os.makedirs(dir_inv)
with open(dir_project+'Basic_info.yml', 'w', encoding='utf-8') as f:
   yaml.dump(data=info_basic, stream=f, allow_unicode=True)

In [16]:
dir_disp = dir_inv + 'data/'
if not os.path.exists(dir_disp):
    os.makedirs(dir_disp)

### Autopick

In [17]:
flag_f = [5,13]

In [18]:
def transfer_autopick(key_subwork):
    global dir_disp_autopick, dir_disp, flag_f
    print(key_subwork)
    ff_pick = np.loadtxt(dir_disp_autopick+'autopick_'+key_subwork+'.txt',delimiter=',')[:,0]
    cc_pick = np.loadtxt(dir_disp_autopick+'autopick_'+key_subwork+'.txt',delimiter=',')[:,1]
    f_start = flag_f[0]
    f_end = flag_f[1]
    ff_this = ff_pick[(ff_pick>=f_start) & (ff_pick<=f_end)]
    cc_this = cc_pick[(ff_pick>=f_start) & (ff_pick<=f_end)]
    mode = np.zeros(len(ff_this))
    np.savetxt(dir_disp+'ds_'+key_subwork+'curve.txt',np.c_[ff_this,cc_this,mode], fmt='%.6f  %.6f  %i')

In [19]:
for key_subwork in key_subworks:
    #print(key_subwork)
    transfer_autopick(key_subwork)

1--08-03
2--09-03


3--10-03
4--11-03
5--12-03
6--13-03
7--14-03
8--15-03
9--16-03
10--18-03
11--19-03
12--20-03
13--21-03
14--22-03
15--23-03
16--24-03
17--25-03
18--26-03
19--27-03
20--28-03
21--29-03
22--30-03
23--31-03
24--32-03
25--33-03
26--34-03
27--35-03
28--36-03
29--37-03
30--41-03
31--42-03
32--43-03
33--46-03
34--47-03
35--48-03
36--52-03
37--53-03
38--07-04
39--08-04
40--09-04
41--10-04
42--12-04
43--13-04
44--17-04
45--20-04
46--21-04
47--22-04
48--23-04
49--24-04
50--25-04
51--27-04
52--28-04
53--29-04
54--30-04
55--31-04
56--32-04
57--33-04
58--34-04
59--35-04
60--36-04
61--41-04
62--42-04
63--47-04
64--48-04
65--49-04
66--52-04
67--53-04
68--08-05
69--12-05
70--13-05
71--14-05
72--16-05
73--17-05
74--19-05
75--22-05
76--24-05
77--26-05
78--27-05
79--28-05
80--29-05
81--30-05
82--32-05
83--33-05
84--35-05
85--36-05
86--37-05
87--38-05
88--39-05
89--40-05
90--41-05
91--42-05
92--43-05
93--44-05
94--48-05
95--49-05
96--51-05
97--52-05
98--53-05
99--54-05
100--07-06
101--08-06
102--10-06
103-